---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [18]:
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')

def blight_model():
    ## Read in the files.
    train = pd.read_csv('train.csv', encoding='ISO-8859-1')
    test = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')

    ## create a single DF with the addresses merged with lat / long.
    addresses = addresses.merge(latlons, how='left', on='address')
    ## Strip off the addresses, just leave the lat / long and ticket_id.
    addresses.drop(['address'], axis=1, inplace=True)
    
    ## Merge the lat / long into the training set (merge by ticket_id)
    train = train.merge(addresses, on='ticket_id', how='left')
    
    ## Drop any items where compliance is null.
    train = train[~train.compliance.isnull()]

    ## Add in the lat / long to the test set.
    test = test.merge(addresses, on='ticket_id', how='left')
    ## Pick the features we care about.
    features = ['violation_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 
                'clean_up_cost', 'judgment_amount', 'grafitti_status', 'lat', 'lon']

    ## Create our X_train (using the features we selected).
    X_train = train[features]
    ## Create our y_train (target - compliance)
    y_train = train['compliance']
    ## Create our X_test (using the same features as the training set)
    X_test = test[features]
    
    ## Create a merged set of our training and test data - for imputing missing data and looking for dummies.
    X_merged = pd.concat([X_train, X_test], axis=0)
    X_merged = pd.get_dummies(X_merged).reset_index(drop=True)

    ## Recreate our train and test sets
    X_train = X_merged.ix[0:X_train.shape[0]-1,:]
    X_test = X_merged.ix[X_train.shape[0]:,:]
        
    ## For all NaNs, grab the mean of the feature.
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)

    ## Train our model.  So far, logistic regression is fast and seems to be fairly accurate.
    #clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.05, max_depth=2).fit(X_train,y_train)
    clf = RandomForestClassifier(n_estimators=300, max_features=6, n_jobs=-1).fit(X_train,y_train)
    #clf = LogisticRegression(C=1000, n_jobs=-1).fit(X_train,y_train)
    
    ## What's our training set accuracy.
    #print("Training set accuracy:  ", clf.score(X_train, y_train))

    ## Make our predictions on the test set.
    proba = pd.Series(clf.predict_proba(X_test)[:,1])
    ## Set the index value (so we know what ticket each prediction goes with)
    proba.index = test['ticket_id']
    
    ## Return the predictions.
    return proba

In [17]:
blight_model()

Training set accuracy:   0.935407805854


ticket_id
284932    0.084624
285362    0.036528
285361    0.066276
285338    0.077030
285346    0.093208
285345    0.077030
285347    0.108129
285342    0.255621
285530    0.044254
284989    0.043417
285344    0.075775
285343    0.041538
285340    0.041538
285341    0.075395
285349    0.093208
285348    0.077030
284991    0.043417
285532    0.042955
285406    0.038730
285001    0.047880
285006    0.049658
285405    0.036767
285337    0.037736
285496    0.064538
285497    0.068063
285378    0.036899
285589    0.038031
285585    0.068114
285501    0.069821
285581    0.037047
            ...   
376367    0.040865
376366    0.044464
376362    0.044844
376363    0.067995
376365    0.040865
376364    0.044464
376228    0.053160
376265    0.043749
376286    0.263053
376320    0.046527
376314    0.041732
376327    0.299972
376385    0.295304
376435    0.230917
376370    0.299972
376434    0.072275
376459    0.105106
376478    0.037143
376473    0.045476
376484    0.042931
376482    0.040314
37